<a href="https://colab.research.google.com/github/leon-czarlinski/ProfileAnalysis/blob/main/ProfileAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
profile_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ProfileAnalysis/customers.csv')

In [5]:
profile_df.head(10)

,custm_id,gender,age,annual_income,spend_score,profession,work_exp,fam_size
0,1,Male,19,15000,39,Healthcare,1,4
1,2,Male,21,35000,81,Engineer,3,3
2,3,Female,20,86000,6,Engineer,1,1
3,4,Female,23,59000,77,Lawyer,0,2
4,5,Female,31,38000,40,Entertainment,2,6
5,6,Female,22,58000,76,Artist,0,2
6,7,Female,35,31000,6,Healthcare,1,3
7,8,Female,23,84000,94,Healthcare,1,3
8,9,Male,64,97000,3,Engineer,0,3
9,10,Female,30,98000,72,Artist,1,4


In [6]:
profile_df.count()

custm_id         2000
gender           2000
age              2000
annual_income    2000
spend_score      2000
profession       1965
work_exp         2000
fam_size         2000
dtype: int64

In [7]:
profile_df.isnull().sum()

custm_id          0
gender            0
age               0
annual_income     0
spend_score       0
profession       35
work_exp          0
fam_size          0
dtype: int64

In [8]:
profile_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   custm_id       2000 non-null   int64 
 1   gender         2000 non-null   object
 2   age            2000 non-null   int64 
 3   annual_income  2000 non-null   int64 
 4   spend_score    2000 non-null   int64 
 5   profession     1965 non-null   object
 6   work_exp       2000 non-null   int64 
 7   fam_size       2000 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 125.1+ KB


In [15]:
null_mask = profile_df.isnull().any(axis=1)
null_rows = profile_df[null_mask]
null_rows

,custm_id,gender,age,annual_income,spend_score,profession,work_exp,fam_size
79,80,Female,49,98000,42,NaN,1,1
118,119,Female,51,84000,43,NaN,2,7
219,220,Female,59,76000,61,NaN,9,1
237,238,Male,95,36000,35,NaN,0,4
437,438,Male,76,136259,14,NaN,0,7
440,441,Female,0,57373,29,NaN,0,7
498,499,Male,95,121725,3,NaN,12,3
545,546,Female,89,107359,26,NaN,10,6
601,602,Male,61,126370,20,NaN,11,4
641,642,Male,66,121377,19,NaN,7,7


In [16]:
profile_df_cp = profile_df.copy()
profile_df_cp

,custm_id,gender,age,annual_income,spend_score,profession,work_exp,fam_size
0,1,Male,19,15000,39,Healthcare,1,4
1,2,Male,21,35000,81,Engineer,3,3
2,3,Female,20,86000,6,Engineer,1,1
3,4,Female,23,59000,77,Lawyer,0,2
4,5,Female,31,38000,40,Entertainment,2,6
...,...,...,...,...,...,...,...,...
1995,1996,Female,71,184387,40,Artist,8,7
1996,1997,Female,91,73158,32,Doctor,7,7
1997,1998,Male,87,90961,14,Healthcare,9,2
1998,1999,Male,77,182109,4,Executive,7,2


In [19]:
profile_df_cp.dropna(inplace=True)
profile_df_cp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1965 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   custm_id       1965 non-null   int64 
 1   gender         1965 non-null   object
 2   age            1965 non-null   int64 
 3   annual_income  1965 non-null   int64 
 4   spend_score    1965 non-null   int64 
 5   profession     1965 non-null   object
 6   work_exp       1965 non-null   int64 
 7   fam_size       1965 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 138.2+ KB
